In [69]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, MaxPooling2D, AveragePooling2D, Conv1D, AveragePooling1D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
%matplotlib inline

In [70]:
def get_random_data(data1, data2, low, high, max_samples=100):
    _, H1, W1 = data1.shape
    N, C = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1))
    suff_data2 = np.zeros((max_samples, C))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :]
        suff_data2[idx] = data2[idx]
    return suff_data1, suff_data2

In [71]:
MAX_TRAINING = 5000
MAX_VALIDATION = 2000
MAX_TESTING = 2000
correct_class = {0: "T-shirt/top", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat", 5: "Sandal", \
                  6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle boot"}

In [72]:
#Loading Fashion-MNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist 
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Keeping the data between 0 and 1
x_train_max, x_test_max = x_train / 255.0, x_test / 255.0
y_train_max = to_categorical(y_train, num_classes=10)
y_test_max = to_categorical(y_test, num_classes=10)

train_cutoff = int(x_train_max.shape[0] - MAX_VALIDATION*1.5)

x_train, y_train = get_random_data(x_train_max, y_train_max, 0, train_cutoff, MAX_TRAINING)
x_val, y_val = get_random_data(x_train_max, y_train_max, train_cutoff+1, x_train_max.shape[0], MAX_VALIDATION)
x_test, y_test = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], MAX_TESTING)

# x_train = np.expand_dims(x_train, axis=1)
# y_train = np.expand_dims(y_train, axis=1)
# x_val = np.expand_dims(x_val, axis=1)
# y_val = np.expand_dims(x_val, axis=1)
# x_test = np.expand_dims(x_test, axis=1)
# y_test = np.expand_dims(y_test, axis=1)

x_train = x_train[..., np.newaxis]
y_train = y_train[..., np.newaxis]
x_val = x_val[..., np.newaxis]
y_val = y_val[..., np.newaxis]
x_test = x_test[..., np.newaxis]
y_test = y_test [..., np.newaxis]

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=500, Y=500
Number of Validation Samples: X=100, Y=100
Number of Test Samples: X=100, Y=100


In [73]:
x_train.shape

(500, 28, 28, 1)

In [74]:
class MetricsAfterEachEpoch(tf.keras.callbacks.Callback):
    def on_train_begin(self, scores={}):
        self.loss, self.accuracy, self.mae, self.mse = [], [], [], []
    def on_train_end(self, logs={}):
        return self.loss, self.accuracy, self.mae, self.loss
    def on_epoch_end(self, epoch, scores):
        self.loss.append(scores.get('loss'))
        self.accuracy.append(scores.get('acc'))
        self.mae.append(scores.get('mean_absolute_error'))
        self.loss.append(scores.get('mean_squared_error'))
        return

In [75]:
def LeNet5Classifier(activation_func):
    # Creating a LeNet5 Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(20, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), name='pool1'))

    # #Instantiating Layer 2
    model.add(Conv2D(50, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))

    # #Instantiating Layer 3
    model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))

    model.add(Flatten())

    #Instantiating Layer 4
    model.add(Dense(500, activation=activation_func)) 

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [76]:
def LeNet5Evaluator(curr_optimizer, activation_func, label=None, curr_epochs=15, saveModel=False):
    model = LeNet5Classifier(activation_func)
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, callbacks=[MetricsAfterEachEpoch()], batch_size=2048).history
    score = model.evaluate(x_test, y_test, batch_size=512)
    #Saving the model
    if (saveModel): 
        if label == 'RAdam':
            model.save("../saved_models/{}_{}_{}.hdf5".format('RAdam', activation_func, "LeNet5"))
        else: 
            model.save("../saved_models/{}_{}_{}.hdf5".format(curr_optimizer, activation_func, "LeNet5"))
    return score, history

In [ ]:
testing_optimizers = ['RAdam', 'Adam', 'NAdam', 'SGD']
testing_activations = ['tanh', 'relu']

for optimizer in testing_optimizers:
    for activation in testing_activations:
        loss, accuracy, mae, mse = {}, {}, {}, {}
        train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict = {}, {}, {}, {}
        
        if optimizer == 'RAdam':
            score_returned, history = LeNet5Evaluator(RAdam(lr=0.0005), activation, label='RAdam')
        else:
            score_returned, history = LeNet5Evaluator(optimizer, activation)
        
        #Test Side Stats
        loss[(optimizer, activation)] = score_returned[0]
        accuracy[(optimizer, activation)] = score_returned[1]
        mae[(optimizer, activation)] = score_returned[2]
        mse[(optimizer, activation)] = score_returned[3]
        
        #Training Side Stats
        train_loss_dict[(optimizer, activation)] = history.get('loss')
        train_accuracy_dict[(optimizer, activation)] = history.get('acc')
        train_mae_dict[(optimizer, activation)] = history.get('mean_absolute_error')
        train_mse_dict[(optimizer, activation)] = history.get('mean_squared_error')
        
        bg = (loss, accuracy, mae, mse, train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict)
        #Delete Existing Primary Keys and then write to DB
        #deleteExistingPrimaryKeyDB(optimizer, activation, 'LeNet5', 'milestone-experiments', 'benchmark_testing')
        #writeToDB(optimizer, activation, 'LeNet5', bg, 'milestone-experiments', 'benchmark_testing')

Train on 500 samples
Epoch 1/15
500/500 [==============================] - 5s 10ms/sample - loss: 2.3153 - accuracy: 0.1340 - mae: 0.1800 - mse: 0.0903
Epoch 2/15
500/500 [==============================] - 1s 1ms/sample - loss: 2.3145 - accuracy: 0.1340 - mae: 0.1800 - mse: 0.0902
Epoch 3/15
500/500 [==============================] - 1s 2ms/sample - loss: 2.3137 - accuracy: 0.1340 - mae: 0.1800 - mse: 0.0902
Epoch 4/15
500/500 [==============================] - 1s 2ms/sample - loss: 2.3129 - accuracy: 0.1360 - mae: 0.1800 - mse: 0.0902
Epoch 5/15
500/500 [==============================] - 1s 2ms/sample - loss: 2.3121 - accuracy: 0.1360 - mae: 0.1800 - mse: 0.0902
Epoch 6/15
500/500 [==============================] - 1s 2ms/sample - loss: 2.3113 - accuracy: 0.1380 - mae: 0.1800 - mse: 0.0902
Epoch 7/15
500/500 [==============================] - 1s 2ms/sample - loss: 2.2999 - accuracy: 0.1520 - mae: 0.1798 - mse: 0.0899
Epoch 8/15
500/500 [==============================] - 1s 1ms/sample 

In [ ]:
print ("Final: Loss={}, Accuracy={}".format(loss, accuracy))